In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from PIL import Image

In [70]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [71]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [72]:
trainset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
testset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=trainset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=testset, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [73]:
model_resnet = torchvision.models.resnet18(pretrained=True)
model_resnet = model_resnet.to(device)

count_classes = 10
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, count_classes).to(device)

model_resnet, model_resnet.fc

c:\Users\ilyae\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ilyae\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [74]:
# freezing weights except the last layer
for param in model_resnet.parameters():
    param.requires_grad = False

# unfreezing weights of the last layer
for param in model_resnet.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet.fc.parameters(), lr=0.001)

In [75]:
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        prediction = model_resnet(inputs)
        loss = criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: [{epoch + 1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch: [1/10], Loss: 2.3428
Epoch: [2/10], Loss: 2.0766
Epoch: [3/10], Loss: 1.9532
Epoch: [4/10], Loss: 1.8853
Epoch: [5/10], Loss: 1.8367
Epoch: [6/10], Loss: 1.8046
Epoch: [7/10], Loss: 1.7752
Epoch: [8/10], Loss: 1.7556
Epoch: [9/10], Loss: 1.7363
Epoch: [10/10], Loss: 1.7231


In [76]:
correct, total = 0, 0

model_resnet.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        prediction = model_resnet(inputs)
        _, predicted = torch.max(prediction, 1)
        total += labels.size(0) # 0 returns 1st size - batch_sizes
        correct += (predicted == labels).sum().item()
print(f"Accuracy on {total} test images: {100 * correct / total}%")

Accuracy on 10000 test images: 40.74%


In [77]:
classes = {
    0: 'airplane',
    1: 'car',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'hourse',
    8: 'ship',
    9: 'truck',
}

In [78]:
image = Image.open("./images/dog2.jpg")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()])
image_tensor = transform(image)

predicted_class = model_resnet(image_tensor.unsqueeze(0).to(device))

with torch.no_grad():
    _, predicted = torch.max(predicted_class, dim=1)
    predicted_class = classes.get(int(predicted))
    
classes.get(int(predicted))

'car'

In [79]:
image = Image.open("./images/dog2.jpg")
image = image.resize((224, 224))
image.show()